In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture   
%pwd


In [3]:
%%capture
cd ..

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import psycopg2 as ps2
import os
import pandas as pd
import numpy as np
import csv
from src.funcs.utils import *

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# pull in db credentials from .env file
db_name=os.getenv('DB_NAME')
db_user=os.getenv('DB_USER')
db_host=os.getenv('DB_HOST_IP')
db_pwd=os.getenv('DB_PWD')

In [7]:
db_host

'10.156.21.123'

In [ ]:
# conn.close()
# conn.rollback()

In [8]:
# define db session
conn = ps2.connect(
    host = db_host,
    database = db_name,
    user = db_user,
    password = db_pwd)

### Relationships

#### Query for data

##### Directional

In [8]:
# define query
# sql="SELECT DISTINCT \
control.id, inkey[1], controltype, ontology, relationship, effect, mechanism, num_refs, outkey[1] , reference.id, biomarkertype, celllinename, celltype, \
changetype, organ, organism, quantitativetype, tissue  \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null"

In [13]:
# define query
sql="SELECT DISTINCT \
control.id, inkey[1], controltype,  string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), \
num_refs, outkey[1] , reference.id, string_agg(distinct(biomarkertype), ', ') , string_agg(distinct(celllinename), ', '), \
string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), string_agg(distinct(organ), ', '), \
string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', '),  \
string_agg(distinct(nct_id), ', '),  string_agg(distinct(phase), ', ') \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null \
GROUP BY control.id, inkey[1], controltype, outkey[1], reference.id"

In [ ]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/relations_query_csl_test.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

##### Bi-derectional

In [8]:
# sql="SELECT DISTINCT \
control.id, inkey[1], inoutkey, controltype, ontology, relationship, effect, mechanism, num_refs, outkey[1] , reference.id, biomarkertype, celllinename, celltype, \
changetype, organ, organism, quantitativetype, tissue  \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is null and outkey[1] is null"

In [24]:
sql="SELECT DISTINCT \
control.id, inkey[1], inoutkey, controltype, relationship, string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), num_refs, outkey[1], \
reference.id, string_agg(distinct(biomarkertype), ', '), string_agg(distinct(celllinename), ', '), string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), \
string_agg(distinct(organ), ', '), string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', ') \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is null and outkey[1] is null \
GROUP BY control.id, controltype, reference.id"

In [45]:
df_rel.columns=['id1', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [25]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/bidirectional_relations_query_csl_test.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

Wall time: 1h 46min 43s


##### Attributes

In [15]:
# sql="SELECT DISTINCT \
id, inkey[1], attributes, relationship, outkey[1] from resnet.control \
WHERE (control.id = control.attributes)";

In [15]:
sql="SELECT DISTINCT \
id, inkey[1], attributes, relationship, outkey[1] from resnet.control \
WHERE (control.id = control.attributes)";

In [16]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/attributes_relations.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

Wall time: 6min 46s


#### Load Data

##### Bi-directional

In [47]:
%%time

df_birect=pd.read_csv('./data/processed/bidirectional_relations_query_csl_test.txt', sep='|', header=None, encoding='utf-8')

df_birect.info(); df_relII.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 18 columns):
0     int64
1     object
2     object
3     object
4     object
5     object
6     object
7     int64
8     object
9     int64
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
dtypes: int64(3), object(15)
memory usage: 648.0+ MB
Wall time: 14.5 s


,id1,:START_ID,inOutkey,type:TYPE,relationship,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,-9223367021478908826,None,"5747547533469447186, 6031869671835980567",FunctionalAssociation,None,None,None,1,None,-9223367021478908826,None,None,None,None,liver,None,None,None
1,-9223365481314032826,None,"5445426109658919917, -2285403010002775161",FunctionalAssociation,None,None,None,1,None,-9223365481314032826,None,None,None,None,leg,None,None,None
2,-9223357200775258657,None,"1596328558605610184, -8708891732207432195",CellExpression,None,None,None,11,None,-9223357200775258657,None,MDA-MB-231,None,None,breast,"Homo sapiens,Oryctolagus cuniculus",None,None
3,-9223356961959519979,None,"4541556823765750071, 3096071376414863663",CellExpression,None,None,None,1,None,-9223356961959519979,None,A2058,None,None,None,None,None,None
4,-9223351835527058603,None,"-2894717825806059007, -5196628532512894922",FunctionalAssociation,None,None,None,2,None,-9223351835527058603,None,None,None,None,thoracolumbar spine,None,None,None


In [48]:
df_birect.columns=['id1', ':START_ID', 'inOutkey', 'type:TYPE', 'relationship', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', 'biomarkertype', 'celllinename', 'celltype',
      'changetype', 'organ', 'organism', 'quantitativetype', 'tissue']

df_birect.head()

,id1,:START_ID,inOutkey,type:TYPE,relationship,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,-9223367021478908826,None,"[5747547533469447186, 6031869671835980567]",FunctionalAssociation,None,None,None,1,None,-9223367021478908826,None,None,None,None,liver,None,None,None
1,-9223365481314032826,None,"[5445426109658919917, -2285403010002775161]",FunctionalAssociation,None,None,None,1,None,-9223365481314032826,None,None,None,None,leg,None,None,None
2,-9223357200775258657,None,"[1596328558605610184, -8708891732207432195]",CellExpression,None,None,None,11,None,-9223357200775258657,None,MDA-MB-231,None,None,breast,"Homo sapiens,Oryctolagus cuniculus",None,None
3,-9223356961959519979,None,"[4541556823765750071, 3096071376414863663]",CellExpression,None,None,None,1,None,-9223356961959519979,None,A2058,None,None,None,None,None,None
4,-9223351835527058603,None,"[-2894717825806059007, -5196628532512894922]",FunctionalAssociation,None,None,None,2,None,-9223351835527058603,None,None,None,None,thoracolumbar spine,None,None,None


In [49]:
first_ids, second_ids = inOutkeys_to_lists(df_birect)

len(first_ids); len(second_ids)
first_ids[:5]; second_ids[:5]

4718418

4718418

['5747547533469447186',
 '5445426109658919917',
 '1596328558605610184',
 '4541556823765750071',
 '-2894717825806059007']

['6031869671835980567',
 '-2285403010002775161',
 '-8708891732207432195',
 '3096071376414863663',
 '-5196628532512894922']

In [50]:
df_birect_1=df_birect.copy()

df_birect_1.drop(columns=['inOutkey', 'id1', 'id2', 'relationship'], inplace=True)

In [51]:
df_birect_1[':START_ID']=first_ids
df_birect_1[':END_ID']=second_ids
df_birect_1.info(); df_rel_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
:START_ID           object
type:TYPE           object
effect              object
mechanism           object
ref_count:int       int64
:END_ID             object
biomarkertype       object
celllinename        object
celltype            object
changetype          object
organ               object
organism            object
quantitativetype    object
tissue              object
dtypes: int64(1), object(13)
memory usage: 504.0+ MB


,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,5747547533469447186,FunctionalAssociation,None,None,1,6031869671835980567,None,None,None,None,liver,None,None,None
1,5445426109658919917,FunctionalAssociation,None,None,1,-2285403010002775161,None,None,None,None,leg,None,None,None
2,1596328558605610184,CellExpression,None,None,11,-8708891732207432195,None,MDA-MB-231,None,None,breast,"Homo sapiens,Oryctolagus cuniculus",None,None
3,4541556823765750071,CellExpression,None,None,1,3096071376414863663,None,A2058,None,None,None,None,None,None
4,-2894717825806059007,FunctionalAssociation,None,None,2,-5196628532512894922,None,None,None,None,thoracolumbar spine,None,None,None


In [52]:
df_birect_1[':START_ID']=df_birect_1[':START_ID'].astype('int64')
df_birect_1[':END_ID']=df_birect_1[':END_ID'].astype('int64')
df_birect_1['ref_count:int']=df_birect_1['ref_count:int'].astype('int8')

df_birect_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
:START_ID           int64
type:TYPE           object
effect              object
mechanism           object
ref_count:int       int8
:END_ID             int64
biomarkertype       object
celllinename        object
celltype            object
changetype          object
organ               object
organism            object
quantitativetype    object
tissue              object
dtypes: int64(2), int8(1), object(11)
memory usage: 472.5+ MB


In [53]:
df_birect_1=convert_object_to_category(df_birect_1)
df_birect_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
:START_ID           int64
type:TYPE           category
effect              category
mechanism           category
ref_count:int       int8
:END_ID             int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
organ               category
organism            category
quantitativetype    category
tissue              category
dtypes: category(11), int64(2), int8(1)
memory usage: 208.0 MB


In [54]:
df_birect_1['biomarkertype'].value_counts()
df_birect_1['mechanism'].value_counts()
df_birect_1['changetype'].value_counts()
df_birect_1['quantitativetype'].value_counts()


None    4718418
Name: biomarkertype, dtype: int64

None       4655608
surface      62810
Name: mechanism, dtype: int64

None    4718418
Name: changetype, dtype: int64

None    4718418
Name: quantitativetype, dtype: int64

In [55]:
df_birect_1.to_pickle('./data/processed/bidirectional_df.pkl')

##### Directional

In [43]:
%%time

df_direct=pd.read_csv('./data/interim/relations_query_csl_test.txt', sep='|', header=None, encoding='utf-8')

df_direct.info(); df_direct.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
0     int64
1     int64
2     object
3     object
4     object
5     int64
6     int64
7     int64
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB
Wall time: 32.1 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [45]:
df_rel.columns=['id1', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [46]:
df_rel.info(); df_rel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
id1                 int64
:START_ID           int64
type:TYPE           object
effect              object
mechanism           object
ref_count:int       int64
:END_ID             int64
id2                 int64
biomarkertype       object
celllinename        object
celltype            object
changetype          object
organ               object
organism            object
quantitativetype    object
tissue              object
nct_id              object
phase               object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB


,id1,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [47]:
df_rel=df_rel.drop(columns=['id1', 'id2'])
df_rel.head()

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,7111680438658927661,Regulation,None,None,1,-3121142714757276759,None,None,None,None,None,None,None,None,None,None
2,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-6414782697893422681,Expression,positive,None,1,-296331965523681380,None,None,T-cell,None,None,None,None,None,None,None


In [51]:
df_rel['phase']=df_rel['phase'].fillna('None')

In [52]:
df_obj = df_rel.select_dtypes(['object'])

for col in list(df_obj.columns):
    df_rel[col] = df_rel[col].apply(lambda x: x.strip())
    df_rel[col]=df_rel[col].astype('category')
    
df_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 16 columns):
:START_ID           int64
type:TYPE           category
effect              category
mechanism           category
ref_count:int       int64
:END_ID             int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
organ               category
organism            category
quantitativetype    category
tissue              category
nct_id              category
phase               category
dtypes: category(13), int64(3)
memory usage: 594.7 MB


In [53]:
df_rel['ref_count:int']=df_rel['ref_count:int'].astype('int8')

In [81]:
df_rel.isnull().sum()

:START_ID           0
type:TYPE           0
effect              0
mechanism           0
ref_count:int       0
:END_ID             0
biomarkertype       0
celllinename        0
celltype            0
changetype          0
organ               0
organism            0
quantitativetype    0
tissue              0
nct_id              0
phase               0
dtype: int64

In [82]:
df_rel['type:TYPE'].value_counts()

Regulation               6310394
Expression                972328
DirectRegulation          780719
QuantitativeChange        503905
GeneticChange             466177
MolTransport              282980
MolSynthesis              179200
StateChange               168537
Biomarker                 145749
ClinicalTrial             128920
ProtModification           80243
miRNAEffect                67012
ChemicalReaction           60486
PromoterBinding            44478
FunctionalAssociation          1
Name: type:TYPE, dtype: int64

In [56]:
df_rel.to_pickle('./data/processed/directional_df.pkl')

##### Attributes

In [92]:
%%time

df_att=pd.read_csv('./data/processed/attributes_relations.txt', sep='|', header=None, encoding='utf-8')

df_att.info(); df_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16009643 entries, 0 to 16009642
Data columns (total 5 columns):
0    int64
1    object
2    int64
3    object
4    object
dtypes: int64(2), object(3)
memory usage: 610.7+ MB
Wall time: 19.8 s


,0,1,2,3,4
0,-9223367021478908826,None,-9223367021478908826,None,None
1,-9223365986387833915,-4129340824583713637,-9223365986387833915,None,-2983484129356197270
2,-9223365556180770043,7111680438658927661,-9223365556180770043,None,-3121142714757276759
3,-9223365481314032826,None,-9223365481314032826,None,None
4,-9223365466130767478,-8447327320654612427,-9223365466130767478,None,-1646880163763186214


In [93]:
df_att.columns=['id1', ':START_ID', 'id2', 'type:TYPE', ':END_ID']
df_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16009643 entries, 0 to 16009642
Data columns (total 5 columns):
id1          int64
:START_ID    object
id2          int64
type:TYPE    object
:END_ID      object
dtypes: int64(2), object(3)
memory usage: 610.7+ MB


In [94]:
df_att['type:TYPE'].value_counts()

None             14917159
is-a              1062691
similar-to          25740
part-of              3325
derivative-of         728
Name: type:TYPE, dtype: int64

In [95]:
df_att=df_att[df_att['type:TYPE']!='None']
df_att.reset_index(drop=True, inplace=True)
df_att. info(); df_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 5 columns):
id1          1092484 non-null int64
:START_ID    1092484 non-null object
id2          1092484 non-null int64
type:TYPE    1092484 non-null object
:END_ID      1092484 non-null object
dtypes: int64(2), object(3)
memory usage: 41.7+ MB


,id1,:START_ID,id2,type:TYPE,:END_ID
0,-9223342075146411186,-4504017200233399253,-9223342075146411186,is-a,5843242504706058497
1,-9223309118206184327,-7802325777421990647,-9223309118206184327,similar-to,2474720540421519282
2,-9223305918769076311,-7394874569383578266,-9223305918769076311,is-a,-1259866436866537965
3,-9223268026487397641,-4464184570445238538,-9223268026487397641,is-a,-7977145807157790077
4,-9223262990187842537,3044332985631925343,-9223262990187842537,is-a,5081693930784726829


In [96]:
df_att=df_att.drop(columns=['id1', 'id2'])
df_att.info(); df_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 3 columns):
:START_ID    1092484 non-null object
type:TYPE    1092484 non-null object
:END_ID      1092484 non-null object
dtypes: object(3)
memory usage: 25.0+ MB


,:START_ID,type:TYPE,:END_ID
0,-4504017200233399253,is-a,5843242504706058497
1,-7802325777421990647,similar-to,2474720540421519282
2,-7394874569383578266,is-a,-1259866436866537965
3,-4464184570445238538,is-a,-7977145807157790077
4,3044332985631925343,is-a,5081693930784726829


In [97]:
for col in list(df_att.columns):
    df_att[col]=df_att[col].apply(lambda x: x.strip())

In [98]:
df_att[':START_ID']=df_att[':START_ID'].astype('int64')
df_att[':END_ID']=df_att[':END_ID'].astype('int64')
df_att['type:TYPE']=df_att['type:TYPE'].astype('category')

df_att.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 3 columns):
:START_ID    1092484 non-null int64
type:TYPE    1092484 non-null category
:END_ID      1092484 non-null int64
dtypes: category(1), int64(2)
memory usage: 17.7 MB


In [99]:
df_att.to_pickle('./data/processed/attributes_df.pkl')

### Concatenate Relationships

In [100]:
df_directional=pd.read_pickle('./data/processed/directional_df.pkl')
df_directional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 16 columns):
:START_ID           int64
type:TYPE           category
effect              category
mechanism           category
ref_count:int       int8
:END_ID             int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
organ               category
organism            category
quantitativetype    category
tissue              category
nct_id              category
phase               category
dtypes: category(13), int64(2), int8(1)
memory usage: 468.6 MB


In [101]:
df_bidirect=pd.read_pickle('./data/processed/bidirectional_df.pkl')
df_bidirect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718418 entries, 0 to 4718417
Data columns (total 14 columns):
:START_ID           int64
type:TYPE           category
effect              category
mechanism           category
ref_count:int       int8
:END_ID             int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
organ               category
organism            category
quantitativetype    category
tissue              category
dtypes: category(11), int64(2), int8(1)
memory usage: 189.0 MB


In [102]:
df_att=pd.read_pickle('./data/processed/attributes_df.pkl')
df_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092484 entries, 0 to 1092483
Data columns (total 3 columns):
:START_ID    1092484 non-null int64
type:TYPE    1092484 non-null category
:END_ID      1092484 non-null int64
dtypes: category(1), int64(2)
memory usage: 17.7 MB


In [103]:
%%time

df_concat=pd.concat([df_directional, df_bidirect, df_att])

C:\Users\Gilbert1\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Wall time: 6.55 s


In [105]:
df_concat.info(); df_concat.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16002031 entries, 0 to 1092483
Data columns (total 16 columns):
:END_ID             int64
:START_ID           int64
biomarkertype       object
celllinename        object
celltype            object
changetype          object
effect              object
mechanism           object
nct_id              object
organ               object
organism            object
phase               object
quantitativetype    object
ref_count:int       float64
tissue              object
type:TYPE           object
dtypes: float64(1), int64(2), object(13)
memory usage: 2.0+ GB


,:END_ID,:START_ID,biomarkertype,celllinename,celltype,changetype,effect,mechanism,nct_id,organ,organism,phase,quantitativetype,ref_count:int,tissue,type:TYPE
0,-2983484129356197270,-4129340824583713637,None,None,None,None,None,None,None,intestine,Rattus norvegicus,None,None,1.0,None,Regulation
1,-3121142714757276759,7111680438658927661,None,None,None,None,None,None,None,None,None,None,None,1.0,None,Regulation
2,-1646880163763186214,-8447327320654612427,None,TOV-112D,"B-cell,endothelial cell",None,None,None,None,peripheral nervous system,None,None,None,10.0,None,Regulation
3,-4650140568030604438,3670031740130349683,None,None,plasma cell,None,positive,None,None,None,None,None,None,1.0,plasma,Regulation
4,-296331965523681380,-6414782697893422681,None,None,T-cell,None,positive,None,None,None,None,None,None,1.0,None,Expression


In [106]:
df_concat.isnull().sum()

:END_ID                   0
:START_ID                 0
biomarkertype       1092484
celllinename        1092484
celltype            1092484
changetype          1092484
effect              1092484
mechanism           1092484
nct_id              5810902
organ               1092484
organism            1092484
phase               5810902
quantitativetype    1092484
ref_count:int       1092484
tissue              1092484
type:TYPE                 0
dtype: int64

In [107]:
df_concat['ref_count:int'].fillna(0, inplace=True)

In [108]:
df_concat=df_concat.fillna('None')


In [109]:
df_concat['ref_count:int']=df_concat['ref_count:int'].astype('int8')

In [110]:
df_obj = df_concat.select_dtypes(['object'])

for col in list(df_obj.columns):
#     df_concat[col]=df_concat[col].astype(str)
    df_concat[col] = df_concat[col].apply(lambda x: x.strip())
    df_concat[col]=df_concat[col].astype('category')
    
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16002031 entries, 0 to 1092483
Data columns (total 16 columns):
:END_ID             int64
:START_ID           int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
effect              category
mechanism           category
nct_id              category
organ               category
organism            category
phase               category
quantitativetype    category
ref_count:int       int8
tissue              category
type:TYPE           category
dtypes: category(13), int64(2), int8(1)
memory usage: 932.6 MB


In [111]:
# df_concat=df_concat[(df_concat[':START_ID']!='None') | (df_concat[':END_ID']!='None')]
df_concat.reset_index(drop=True, inplace=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16002031 entries, 0 to 16002030
Data columns (total 16 columns):
:END_ID             int64
:START_ID           int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
effect              category
mechanism           category
nct_id              category
organ               category
organism            category
phase               category
quantitativetype    category
ref_count:int       int8
tissue              category
type:TYPE           category
dtypes: category(13), int64(2), int8(1)
memory usage: 810.5 MB


In [113]:
df_concat['celllinename'].value_counts()
df_concat['celltype'].value_counts()

df_concat['biomarkertype'].value_counts()
df_concat['mechanism'].value_counts()
df_concat['changetype'].value_counts()
df_concat['quantitativetype'].value_counts()
df_concat['effect'].value_counts()
df_concat['tissue'].value_counts()
df_concat['organism'].value_counts()

None                                      14103009
HEK293                                       51472
HepG 2                                       48451
293T                                         37567
A 549                                        34821
                                            ...   
DLD I,RAW264.7,SGC 7901,VCAP                     1
DLD I,RAJI,SKHep1                                1
DLD I,QGY-7703                                   1
DLD I,QBC939                                     1
C3H/10T1/2 clone 8,MDA-MB-231,RAW264.7           1
Name: celllinename, Length: 230489, dtype: int64

None                                                                                                                                                                                                                                       10490274
neuron                                                                                                                                                                                                                                       380890
macrophage                                                                                                                                                                                                                                   191863
fibroblast                                                                                                                                                                                                                                   136802
T-cell                  

None                     15862179
diagnostic                  70971
prognostic                  49620
diagnostic,prognostic       19261
Name: biomarkertype, dtype: int64

None                  15086506
direct interaction      780719
surface                  62810
phosphorylation          37909
cleavage                 17983
dephosphorylation         4393
ubiquitination            4305
methylation               2586
acetylation               2510
deacetylation             1479
demethylation              395
deubiquitination           216
sumoylation                195
desumoylation               22
export                       2
degradation                  1
Name: mechanism, dtype: int64

None                                                                15678033
mutation                                                              255441
phosphorylation                                                        17743
epigenetic methylation                                                 11149
gene amplification                                                     10274
alternative splicing                                                    6103
gene deletion                                                           5855
gene deletion,mutation                                                  5657
gene amplification,mutation                                             5279
epigenetic methylation,mutation                                         3484
epigenetic methylation,gene deletion,mutation                            643
gene amplification,gene deletion,mutation                                637
epigenetic methylation,gene amplification,mutation                       486

None                                       15530886
expression                                   167880
abundance                                    110796
activity                                      82168
secretion                                     33272
abundance,expression                          31671
activity,expression                           15509
abundance,activity,expression                 11412
abundance,activity                             7848
abundance,secretion                            4547
expression,secretion                           2322
abundance,expression,secretion                 1831
abundance,activity,expression,secretion        1055
activity,expression,secretion                   509
activity,secretion                              233
abundance,activity,secretion                     92
Name: quantitativetype, dtype: int64

None        10322025
negative     2896768
positive     2783238
Name: effect, dtype: int64

None                                                                                                                                                    12786657
plasma                                                                                                                                                    257770
blood                                                                                                                                                     206127
endothelium                                                                                                                                               131415
epithelium                                                                                                                                                119247
                                                                                                                                                          ...   
cartilage,endothelium,epithelium,m

None                                                                                                                                       10008868
Homo sapiens                                                                                                                                2927546
Mus musculus                                                                                                                                1064924
Rattus norvegicus                                                                                                                            654348
Homo sapiens,Mus musculus                                                                                                                    295090
                                                                                                                                             ...   
Caenorhabditis elegans,Drosophila melanogaster,Homo sapiens,Mammalia,Mus musculus,Rattus norvegicus,Saccharomyce

In [114]:
df_concat.drop_duplicates(inplace=True)
df_concat.reset_index(drop=True, inplace=True)

df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001720 entries, 0 to 16001719
Data columns (total 16 columns):
:END_ID             int64
:START_ID           int64
biomarkertype       category
celllinename        category
celltype            category
changetype          category
effect              category
mechanism           category
nct_id              category
organ               category
organism            category
phase               category
quantitativetype    category
ref_count:int       int8
tissue              category
type:TYPE           category
dtypes: category(13), int64(2), int8(1)
memory usage: 810.5 MB


In [115]:
df_concat['type:TYPE'].value_counts()
df_concat['organ'].value_counts()

Regulation               6310394
FunctionalAssociation    2068008
CellExpression           1492256
Binding                  1157797
is-a                     1062397
Expression                972328
DirectRegulation          780719
QuantitativeChange        503905
GeneticChange             466177
MolTransport              283319
MolSynthesis              179200
StateChange               168537
Biomarker                 145749
ClinicalTrial             128920
ProtModification           80243
miRNAEffect                67012
ChemicalReaction           60489
PromoterBinding            44478
similar-to                 25740
part-of                     3325
derivative-of                727
Name: type:TYPE, dtype: int64

None                                                                                                                             9662169
liver                                                                                                                             263333
brain                                                                                                                             246320
heart                                                                                                                             156469
skin                                                                                                                              128663
                                                                                                                                  ...   
bronchus muscle,cerebral cortex,skin,vibrissa                                                                                          1
bronchus muscle,cerebral cortex,nervous s

In [116]:
df_concat=df_concat.replace('None', '_')
df_concat.info(); df_concat.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001720 entries, 0 to 16001719
Data columns (total 16 columns):
:END_ID             int64
:START_ID           int64
biomarkertype       object
celllinename        object
celltype            object
changetype          object
effect              object
mechanism           object
nct_id              object
organ               object
organism            object
phase               object
quantitativetype    object
ref_count:int       int64
tissue              object
type:TYPE           object
dtypes: int64(3), object(13)
memory usage: 1.9+ GB


,:END_ID,:START_ID,biomarkertype,celllinename,celltype,changetype,effect,mechanism,nct_id,organ,organism,phase,quantitativetype,ref_count:int,tissue,type:TYPE
0,-2983484129356197270,-4129340824583713637,_,_,_,_,_,_,_,intestine,Rattus norvegicus,_,_,1,_,Regulation
1,-3121142714757276759,7111680438658927661,_,_,_,_,_,_,_,_,_,_,_,1,_,Regulation
2,-1646880163763186214,-8447327320654612427,_,TOV-112D,"B-cell,endothelial cell",_,_,_,_,peripheral nervous system,_,_,_,10,_,Regulation
3,-4650140568030604438,3670031740130349683,_,_,plasma cell,_,positive,_,_,_,_,_,_,1,plasma,Regulation
4,-296331965523681380,-6414782697893422681,_,_,T-cell,_,positive,_,_,_,_,_,_,1,_,Expression
5,8609728222254108109,-6068746946264226305,_,_,"fibroblast,neuron",_,_,_,_,"face,placenta,prosencephalon,skin",Mus musculus,_,_,10,blood,Regulation
6,-7927185763708446597,5304163336353694133,_,"SH-SY5Y,U-2 OS",microglia,_,positive,_,_,_,_,_,_,4,_,Regulation
7,-7020171786407163262,-9100487092324496351,_,MDA-MB-231,_,_,negative,_,_,_,_,_,_,1,_,Regulation
8,-2766563825486997894,1077676061581638866,_,_,osteoclast,_,_,_,_,bone,_,_,_,1,_,Regulation
9,-8252144629510342204,2080107145996717982,_,_,_,_,_,_,_,_,_,_,_,1,_,Regulation


In [117]:
df_concat['type:TYPE'].replace('-', '_', inplace=True, regex=True)


In [118]:
# df cleanup

df_concat['type:TYPE']=df_concat['type:TYPE'].apply(lambda x: x.upper())
df_concat.info(); df_concat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001720 entries, 0 to 16001719
Data columns (total 16 columns):
:END_ID             int64
:START_ID           int64
biomarkertype       object
celllinename        object
celltype            object
changetype          object
effect              object
mechanism           object
nct_id              object
organ               object
organism            object
phase               object
quantitativetype    object
ref_count:int       int64
tissue              object
type:TYPE           object
dtypes: int64(3), object(13)
memory usage: 1.9+ GB


,:END_ID,:START_ID,biomarkertype,celllinename,celltype,changetype,effect,mechanism,nct_id,organ,organism,phase,quantitativetype,ref_count:int,tissue,type:TYPE
0,-2983484129356197270,-4129340824583713637,_,_,_,_,_,_,_,intestine,Rattus norvegicus,_,_,1,_,REGULATION
1,-3121142714757276759,7111680438658927661,_,_,_,_,_,_,_,_,_,_,_,1,_,REGULATION
2,-1646880163763186214,-8447327320654612427,_,TOV-112D,"B-cell,endothelial cell",_,_,_,_,peripheral nervous system,_,_,_,10,_,REGULATION
3,-4650140568030604438,3670031740130349683,_,_,plasma cell,_,positive,_,_,_,_,_,_,1,plasma,REGULATION
4,-296331965523681380,-6414782697893422681,_,_,T-cell,_,positive,_,_,_,_,_,_,1,_,EXPRESSION


In [119]:
df_concat['type:TYPE'].value_counts()

REGULATION               6310394
FUNCTIONALASSOCIATION    2068008
CELLEXPRESSION           1492256
BINDING                  1157797
IS_A                     1062397
EXPRESSION                972328
DIRECTREGULATION          780719
QUANTITATIVECHANGE        503905
GENETICCHANGE             466177
MOLTRANSPORT              283319
MOLSYNTHESIS              179200
STATECHANGE               168537
BIOMARKER                 145749
CLINICALTRIAL             128920
PROTMODIFICATION           80243
MIRNAEFFECT                67012
CHEMICALREACTION           60489
PROMOTERBINDING            44478
SIMILAR_TO                 25740
PART_OF                     3325
DERIVATIVE_OF                727
Name: type:TYPE, dtype: int64

In [ ]:
# filter ref count =0 if type not Clinical trial

# import numpy as np
# idx =np.where((df_rel_deduped['ref_count:int']==0) & (df_rel_deduped['type:TYPE']!='CLINICALTRIAL') & \
#               (df_rel_deduped['type:TYPE']!='IS-A') & (df_rel_deduped['type:TYPE']!='PART-OF') & \
#              (df_rel_deduped['type:TYPE']!='DERIVATIVE-OF') & (df_rel_deduped['type:TYPE']!='SIMILAR-TO'))  #get index values to filter out
# df_rel_filtered=df_rel_deduped[~df_rel_deduped.index.isin(idx[0])]
# df_rel_filtered.info(); df_rel_filtered.head()

In [121]:
cols=list(df_directional.columns)
cols

[':START_ID',
 'type:TYPE',
 'effect',
 'mechanism',
 'ref_count:int',
 ':END_ID',
 'biomarkertype',
 'celllinename',
 'celltype',
 'changetype',
 'organ',
 'organism',
 'quantitativetype',
 'tissue',
 'nct_id',
 'phase']

In [122]:
df_concat=df_concat[cols]
df_concat.head(5)

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-4129340824583713637,REGULATION,_,_,1,-2983484129356197270,_,_,_,_,intestine,Rattus norvegicus,_,_,_,_
1,7111680438658927661,REGULATION,_,_,1,-3121142714757276759,_,_,_,_,_,_,_,_,_,_
2,-8447327320654612427,REGULATION,_,_,10,-1646880163763186214,_,TOV-112D,"B-cell,endothelial cell",_,peripheral nervous system,_,_,_,_,_
3,3670031740130349683,REGULATION,positive,_,1,-4650140568030604438,_,_,plasma cell,_,_,_,_,plasma,_,_
4,-6414782697893422681,EXPRESSION,positive,_,1,-296331965523681380,_,_,T-cell,_,_,_,_,_,_,_


In [123]:
%%time
df_concat.to_csv('./data/processed/relations.txt', sep='|', index=False, header=True)

Wall time: 2min 30s


In [124]:
# list of column names for header file
cols=list(df_concat.columns)
cols

[':START_ID',
 'type:TYPE',
 'effect',
 'mechanism',
 'ref_count:int',
 ':END_ID',
 'biomarkertype',
 'celllinename',
 'celltype',
 'changetype',
 'organ',
 'organism',
 'quantitativetype',
 'tissue',
 'nct_id',
 'phase']

In [125]:
# create headers file
df_rel_headers=pd.DataFrame(columns=cols)
df_rel_headers
df_rel_headers.to_csv('./data/processed/relations_header.txt', sep= '|', index=False)

,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase


### Nodes

In [126]:
# define query
sql='select id, name, nodetype from resnet.node where id is not null and name is not null and nodetype is not null'

In [127]:
%%time

# save query results to file
with conn.cursor() as cur:
    with open('./data/raw/nodes_.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter='|')
        for record in cur.fetchall():
            line="|".join(map(str, record))+'\n'
            
            f.write(line)

Wall time: 26.8 s


#### Create Node files

In [128]:
df_node=pd.read_csv('./data/raw/nodes_.txt', delimiter='|', header=None, encoding='utf-8')
df_node.info(); df_node.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470805 entries, 0 to 1470804
Data columns (total 3 columns):
0    1470805 non-null int64
1    1470805 non-null object
2    1470805 non-null object
dtypes: int64(1), object(2)
memory usage: 33.7+ MB


,0,1,2
0,1000020775393047472,r_LOC108352567,Protein
1,-1000022879358845591,Rx26449263,SmallMol
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SemanticConcept
3,1000026386881640747,rs876660037,GeneticVariant
4,-1000028657865034959,m_Fbxw19,Protein


In [129]:
# create column headers
df_node.columns=[':ID', 'name', ':LABEL']
df_node.head()

,:ID,name,:LABEL
0,1000020775393047472,r_LOC108352567,Protein
1,-1000022879358845591,Rx26449263,SmallMol
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SemanticConcept
3,1000026386881640747,rs876660037,GeneticVariant
4,-1000028657865034959,m_Fbxw19,Protein


In [130]:
# from start_ID in relationships
df_node[df_node[':ID']==-4966241110786578276]

,:ID,name,:LABEL
704733,-4966241110786578276,Ras GTPase,FunctionalClass


In [131]:
# df cleanup
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.upper())
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.strip())
df_node['name']=df_node['name'].apply(lambda x: x.strip())
df_node[':ID']=df_node[':ID'].apply(lambda x: str(x).strip())  #convert to string and strip()
df_node[':ID']=df_node[':ID'].astype('int64')   # convert back to int
df_node.info(); df_node.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470805 entries, 0 to 1470804
Data columns (total 3 columns):
:ID       1470805 non-null int64
name      1470805 non-null object
:LABEL    1470805 non-null object
dtypes: int64(1), object(2)
memory usage: 33.7+ MB


,:ID,name,:LABEL
0,1000020775393047472,r_LOC108352567,PROTEIN
1,-1000022879358845591,Rx26449263,SMALLMOL
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SEMANTICCONCEPT
3,1000026386881640747,rs876660037,GENETICVARIANT
4,-1000028657865034959,m_Fbxw19,PROTEIN


In [132]:
# check for name strings that are too long
df_node['name_len']=df_node['name'].apply(lambda x: len(x))

In [133]:
df_node.sort_values(by='name_len', ascending=False)

,:ID,name,:LABEL,name_len
1107832,-7235442027224814239,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...,SMALLMOL,283582
600508,-4382490639255431495,"D-Glucose, O-4,6-dideoxy-4-((4-(alpha-D-glucop...",SMALLMOL,254
7778,1043483757283375332,"2,4a-Dihydroxy-1-methyl-8-methylenegibb-3-ene-...",SMALLMOL,248
1229720,7946627188054156171,"Cholan-24-oic acid, 3,7,12-trioxo-, (5beta)-, ...",SMALLMOL,243
377137,-3119824026281474218,"1,3,5-Triazine-2,4,6-triamine, N,N'''-1,2-etha...",SMALLMOL,216
...,...,...,...,...
205050,2153331758759806501,HN,CELLTYPE,2
1086727,-7115660023470151832,c,SMALLMOL,1
1273144,-8190334285627176943,X,SMALLMOL,1
432989,-3437747827402286845,\,SMALLMOL,1


In [134]:
# Issue is leading quote with no ending quote in sql db for id -7235442027224814239 above
# the following is a temp fix unitil resolved in sql db
# process long name field

x=df_node[df_node[':ID']==-7235442027224814239]['name']
long_index=df_node[df_node[':ID']==-7235442027224814239].index[0] #capture index to delete later
long_index
x=x.to_frame().reset_index()
x.info(); x.head()
y=x.loc[0,'name']
len(y)

1107832

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
index    1 non-null int64
name     1 non-null object
dtypes: int64(1), object(1)
memory usage: 144.0+ bytes


,index,name
0,1107832,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...


283582

In [135]:
# parse bad field
lst=y.split('\r\n')
lst[:5]
lst[0]='-7235442027224814239|' + lst[0]
lst[:5]

['1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-1-yl)butyl)-4-(4-fluorophenyl)piperidine-2,6-dione|SmallMol',
 '7235446264322303181|Rx25148080|SmallMol',
 '7235446778373378569|6-chloro-2-fluoropurine|SmallMol',
 '-7235449901734945744|2,2,7-trimethyl-4H-1,4-benzoxazin-3-one|SmallMol',
 '-7235456322816637391|N-(p-Tolyl)-p-toluenesulphonamide|SmallMol']

['-7235442027224814239|1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-1-yl)butyl)-4-(4-fluorophenyl)piperidine-2,6-dione|SmallMol',
 '7235446264322303181|Rx25148080|SmallMol',
 '7235446778373378569|6-chloro-2-fluoropurine|SmallMol',
 '-7235449901734945744|2,2,7-trimethyl-4H-1,4-benzoxazin-3-one|SmallMol',
 '-7235456322816637391|N-(p-Tolyl)-p-toluenesulphonamide|SmallMol']

In [136]:
# create list of lists from long name field
new_list=[]
for i in range(len(lst)):
    z=lst[i].split('|')
    new_list.append(z)
    

In [137]:
len(new_list)
new_list[:5]

3855

[['-7235442027224814239',
  '1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-1-yl)butyl)-4-(4-fluorophenyl)piperidine-2,6-dione',
  'SmallMol'],
 ['7235446264322303181', 'Rx25148080', 'SmallMol'],
 ['7235446778373378569', '6-chloro-2-fluoropurine', 'SmallMol'],
 ['-7235449901734945744',
  '2,2,7-trimethyl-4H-1,4-benzoxazin-3-one',
  'SmallMol'],
 ['-7235456322816637391', 'N-(p-Tolyl)-p-toluenesulphonamide', 'SmallMol']]

In [138]:
# create df from list of lists
df_temp=pd.DataFrame(new_list, columns=[':ID', 'name', ':LABEL'])
df_temp.info(); df_temp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3855 entries, 0 to 3854
Data columns (total 3 columns):
:ID       3855 non-null object
name      3855 non-null object
:LABEL    3854 non-null object
dtypes: object(3)
memory usage: 90.5+ KB


,:ID,name,:LABEL
0,-7235442027224814239,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...,SmallMol
1,7235446264322303181,Rx25148080,SmallMol
2,7235446778373378569,6-chloro-2-fluoropurine,SmallMol
3,-7235449901734945744,"2,2,7-trimethyl-4H-1,4-benzoxazin-3-one",SmallMol
4,-7235456322816637391,N-(p-Tolyl)-p-toluenesulphonamide,SmallMol


In [139]:
df_temp.dropna(inplace=True)

In [140]:
df_temp[':LABEL']=df_temp[':LABEL'].apply(lambda x: x.upper())
df_temp.head()

,:ID,name,:LABEL
0,-7235442027224814239,1-(4-(4-(6-fluorobenzoisoxazol-3-yl)piperidin-...,SMALLMOL
1,7235446264322303181,Rx25148080,SMALLMOL
2,7235446778373378569,6-chloro-2-fluoropurine,SMALLMOL
3,-7235449901734945744,"2,2,7-trimethyl-4H-1,4-benzoxazin-3-one",SMALLMOL
4,-7235456322816637391,N-(p-Tolyl)-p-toluenesulphonamide,SMALLMOL


In [141]:
# combine dfs
df_node.drop(int(f'{long_index}'), inplace = True) #drop row with long name field
df_node.drop(columns=['name_len'], inplace=True)
df_node_new=pd.concat([df_node, df_temp])
df_node_new.reset_index(inplace=True, drop=True)
df_node_new.info(); df_node_new.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474658 entries, 0 to 1474657
Data columns (total 3 columns):
:ID       1474658 non-null object
name      1474658 non-null object
:LABEL    1474658 non-null object
dtypes: object(3)
memory usage: 33.8+ MB


,:ID,name,:LABEL
0,1000020775393047472,r_LOC108352567,PROTEIN
1,-1000022879358845591,Rx26449263,SMALLMOL
2,-1000024917554356200,GO: extracellular matrix of synaptic cleft,SEMANTICCONCEPT
3,1000026386881640747,rs876660037,GENETICVARIANT
4,-1000028657865034959,m_Fbxw19,PROTEIN


In [142]:
# check for presence of ';;' & ';'
df_node_new[df_node_new['name'].str.contains(";")]
df_node_new[df_node_new['name'].str.contains(";;")]

,:ID,name,:LABEL
94,-1000445907920769387,Ac-FRPX(Hle; D)F(3-Cl)F-NH2,SMALLMOL
103,1000511471434219628,(S)-3-chloro-4-(N-(3-fluoropropyl)pyrrolidin-3...,SMALLMOL
1038,10055389442503362,Carbamoyl-C(D;bridge C11)KFFW(D)F(4-(Isopropyl...,SMALLMOL
1301,-1006961062563094732,Ac-ED(lactam K7)HX(2-Nal; D)RWK-NH2,SMALLMOL
1503,1008036727165805856,X(Iqc; D)C(bridge X7)YW(D)RTX(Pen)T-NH2,SMALLMOL
...,...,...,...
1473596,7251653840688617755,X(Inp)F(D;4-(Chloro))X(2-Nal;D)F-Amino,SMALLMOL
1473919,-7253500055656412392,m_T(3;10)61H (Gene ID 21159),PROTEIN
1474113,7254604792324956166,YF(D)GW(D)X(Nle; N-Me)DF-NH2,SMALLMOL
1474252,725545692487216447,"2(S)2-{[6-(5-chloro-1H-pyrrolo[2,3-b]pyridin-3...",SMALLMOL


,:ID,name,:LABEL
8757,-1048773164875225821,"Y(2,6-Me; N-CO)X(Orn(D)modifixfffd;; lactam G5...",SMALLMOL
946734,6327831653333845569,X(Orn(D)modifixfffd;; lactam G4)X(2-Nal)P(D)G,SMALLMOL


In [143]:
# convert single and double semi-colons to colons
df_node_new['name']=df_node_new['name'].str.replace(';;', ':')
df_node_new['name']=df_node_new['name'].str.replace(';', ':')


In [144]:
df_node_new.to_csv('./data/processed/nodes.txt', sep='|', index=False, header=True)

In [145]:
# create list of column names for header file
cols=list(df_node.columns)
cols

[':ID', 'name', ':LABEL']

In [146]:
# create and save header file
node_headers=pd.DataFrame(columns=cols)
node_headers
node_headers.to_csv('./data/processed/nodes_header.txt', sep='|', index=False)

,:ID,name,:LABEL


In [50]:
conn.close()


**To load data into Neo4j**, click on 'Add' in the project pane. Name the new project, click on the '...' by the 'Open' button for the project, and choose 'Terminal'. Take note of the dbms number at the cursor. Copy the nodes.txt, nodes_header.txt, relations.txt, and realations_header.txt files pepared above. Drop them in the import file found here: C:\Users\\[user_name]\\.Neo4jDesktop\relate-data\dbmss\dbms-##(number at cursor)\import

At the Neo4j terminal, change to the bin directory (type 'cd bin') and then paste the following at the cursor:
neo4j-admin import --delimiter="|" --nodes=import/nodes_header.txt, import/nodes.txt --relationships=import/relations_header.txt, import/relations.txt --skip-bad-relationships=true

Go back to the project pane, press 'Start'. The active database will now be shown in a pane at the top. Press open to use in the Neo4j browser.